In [7]:
import matplotlib
matplotlib.use('agg')

import os
import pickle
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
from keras.layers import Input, Convolution2D, Convolution1D, MaxPooling2D, Dense, Dropout, \
                          Flatten, concatenate, Activation, Reshape, \
                          UpSampling2D,ZeroPadding2D
from keras import layers

import keras
from pylab import plt
import tensorflow as tf
from matplotlib.pyplot import plot, figure
from dask.diagnostics import ProgressBar
ProgressBar().register()


import xarray as xr
import numpy as np
import cmocean # for perceptually uniform colormaps
import cartopy as cr # for geographic mapping
import cartopy.crs as ccrs # for map projections
import matplotlib.pyplot as plt # plotting tool
import cartopy.feature as cfeature # to add coastlines, land and ocean
from cartopy import config
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfea
from netCDF4 import Dataset
import matplotlib.pylab as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib as mpl
import datetime
import os
from datetime import datetime
import matplotlib.patheffects as PathEffects
import geocat.viz as gv
import cmaps
import geocat.datafiles as gdf
import geocat.viz as gv
import geocat.viz.util as gvutil
from scipy import stats
from mpl_toolkits.basemap import Basemap, cm, shiftgrid, addcyclic
from scipy import signal
from scipy import stats
import math
import iris
from datetime import datetime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from scipy import stats
import numpy as np
from scipy import signal
import numpy.polynomial.polynomial as poly
from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap
from eofs.standard import Eof
import pint
import pint_pandas

In [2]:
#Saved path
outdir='C:/Users/user/Research/Research Code/CNN/trainning/channelwise_Nonnormalised/running mean/train_1188/'
ifile_data ='C:/Users/user/Research/Research Code/CNN/Data/'
testsizepath='C:/Users/user/Research/Research Code/CNN/y_test/running_mean/'

ifile ='C:/Users/user/Research/Research Code/CNN/trainning/channelwise_Nonnormalised/running mean/train_1428/'
outdir_original_pred = 'C:/Users/user/Research/Research Code/CNN/CW_PDO/original/pred/'
outdir_original_test = 'C:/Users/user/Research/Research Code/CNN/CW_PDO/original/test/'
outdir_original_valid = 'C:/Users/user/Research/Research Code/CNN/CW_PDO/original/validation/'
outdir_smooth_pred = 'C:/Users/user/Research/Research Code/CNN/CW_PDO/smoothed/pred/'
outdir_smooth_test= 'C:/Users/user/Research/Research Code/CNN/CW_PDO/smoothed/test/'
outdir_smooth_valid= 'C:/Users/user/Research/Research Code/CNN/CW_PDO/smoothed/validation/'
outdir_correlation = 'C:/Users/user/Research/Research Code/CNN/CW_PDO/correlation/'
outdir_smooth_pred_label = 'C:/Users/user/Research/Research Code/CNN/CW_PDO/smoothed/pred_label/'
save_path = 'C:/Users/user/Research/Research Code/CNN/PDO/Heatmap/'
outdir =  'C:/Users/user/Research/Research Code/CNN/PDO/'

In [1]:
##Set up all functions
def weighted_areamean(ds):
    '''area mean weighted by cos of latitude'''
    weights = xr.ufuncs.cos(np.deg2rad(ds.lat))
    norm = np.sum(weights) * len(ds.lon)
    amean = (ds*weights).sum(('lat','lon')) / norm

    return amean


def acc_score(x,y):
    '''timestepwise anomaly correlation coefficient, averaged over time
        (simple version without seasonal climatoloty)'''
    assert(x.shape==y.shape)
    return np.mean([np.corrcoef(x[i].flatten(),y[i].flatten())[0,1] for i in range(len(x))])


def corr_over_time(x,y):
    '''timestepwise anomaly correlation coefficient, averaged over time
        (simple version without seasonal climatoloty)'''    
    mx = x[0:i].mean()
    my = y[0:i].mean()
    xm, ym = x[0:i]-mx, y[0:i]-my
    r_num = (xm*ym).mean()
    r_den = xm.std() * ym.std()
    r = r_num / r_den
        
    return r

def mean(x):
    avg = sum(x)/len(x)
    return round(avg,2)


def adjust(ds):
    '''transform the data'''
    z1 = np.array(ds)
    df = pd.DataFrame (z1)
    pred = df.iloc[:,0]
    return pred 

def correlation(x,y,z):
    corr_test = np.corrcoef(x [0:i],y [0:i])[0,1]
    z.append(corr_test)
    
def lowpass_filter (x,y):
    '''fraction of nyquist frequency, here  it is 10 years'''
    fs=1/12/30/24/3600 

    nyquist = fs / 2 # 0.5 times the sampling frequency
    cutoff=x # fraction of nyquist frequency, here  it is 10 years
    b, a = signal.butter(5, cutoff, btype='lowpass') #low pass filter


    dUfilt = signal.filtfilt(b, a, y)
    dUfilt=np.array(dUfilt)
    dUfilt=dUfilt.transpose()
    return dUfilt

def avg(myArray, N=12):
    '''average every 12 months'''
    cum = np.cumsum(myArray,0)
    result = cum[N-1::N]/float(N)
    result[1:] = result[1:] - result[:-1]

    remainder = myArray.shape[0] % N
    if remainder != 0:
        if remainder < myArray.shape[0]:
            lastAvg = (cum[-1]-cum[-1-remainder])/float(remainder)
        else:
            lastAvg = cum[-1]/float(remainder)
        result = np.vstack([result, lastAvg])

    return result

def detrend_dim(da,trend, dim, deg=1):
    '''detrend along a single dimension'''
    p = da.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(trend[dim], p.polyfit_coefficients)
    return da - fit

In [3]:
os.system('mkdir -p '+outdir)
N_gpu = 0
print('open inputdata')

open inputdata


# Start training model

In [6]:
#Imported data
ds = xr.open_dataset(ifile_data+'nonnorm_Datapreprcess_ersst_1854_2022.nc')
ds

<xarray.Dataset>
Dimensions:  (time: 2028, lat: 32, lon: 64, lev: 1)
Coordinates:
  * time     (time) datetime64[ns] 1854-01-01 1854-02-01 ... 2022-12-01
  * lat      (lat) float32 64.0 62.0 60.0 58.0 56.0 ... 10.0 8.0 6.0 4.0 2.0
  * lon      (lon) float32 122.0 124.0 126.0 128.0 ... 242.0 244.0 246.0 248.0
  * lev      (lev) float64 0.0
Data variables:
    sst      (time, lat, lon, lev) float32 ...

In [7]:
##Prepare data
ds=ds['sst'].data 
ds[ds<0]=0 #treat nan values as 0
np.nan_to_num(ds,copy=False)
ds=ds/(np.nanmax(ds)) 
dates=pd.date_range(start='1854-01-01',periods=len(ds))
label=np.array(dates.month)
label=label-1

In [8]:
##Data separation
## Different  models are trained for different target months 
## then All of the model output is put together to make the prediction of PDO index from 1983 to 2022.

N_total = 2028
N_train = 1428
N_gab_1 = 0
N_test = 1
N_skip = N_total-N_train-N_gab_1-N_test-480

x = ds

In [11]:
##Verify that we have sufficient information for the requirements.
if N_skip+N_train+N_gab_1+N_test > x.shape[0]:
    raise Exception('not enough timesteps in input file!')

x = x.astype('float32')
x = x[:N_skip+N_train+N_gab_1+N_test+480]

lat,lon,lev=x.shape[1:4]

In [12]:
def prepare_data(lead_time):
    ''' split up data in predictor and predictant set by shifting
     it according to the given lead time, and then split up
     into train, developement and test set'''
    if lead_time == 0:
        X = x
        y = X[:]

    else:

        X = x[:]
        y = x[:]
    

    X_train = X[N_skip+i-lead_time:N_skip+N_train+i-lead_time]
    y_train = y[N_skip+i-lead_time:N_skip+N_train+i-lead_time]
    
    
    X_test = X[N_skip+N_train+N_gab_1+i:N_skip+N_train+N_gab_1+N_test+i]
    y_test = y[N_skip+N_train+N_gab_1+i:N_skip+N_train+N_gab_1+N_test+i]
    
       
    return X_train,y_train, X_test, y_test

In [13]:
# fixed (not-tuned params)
batch_size = 32
num_epochs = 10
pool_size = 1
drop_prob=0
conv_activation='relu'

In [14]:
def acc_score(x,y):
    '''timestepwise anomaly correlation coefficient, averaged over time
        (simple version without seasonal climatoloty)'''
    assert(x.shape==y.shape)
    return np.mean([np.corrcoef(x[i].flatten(),y[i].flatten())[0,1] for i in range(len(x))])

In [15]:
def build_model(conv_depth, kernel_size, hidden_size, n_hidden_layers, lr):

    model = keras.Sequential([
            
            ## Convolution which involves dimensionality reduction (similar to Encoder in an autoencoder)
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation, input_shape=(lat,lon,lev)),
            layers.MaxPooling2D(pool_size=pool_size),
            Dropout(drop_prob),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            layers.MaxPooling2D(pool_size=pool_size),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            layers.MaxPooling2D(pool_size= 1),
        
        
            # end "encoder"
            
            
            # dense layers (Automatic flattening and reshaping occurs.)
            ] + [layers.Dense(hidden_size, activation='sigmoid') for i in range(n_hidden_layers)] +
             
            [
            
            
            # start "Decoder" (upsampling of the encoder above)
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            layers.UpSampling2D(size=pool_size),
            Convolution2D(conv_depth, kernel_size, padding='same', activation=conv_activation),
            layers.UpSampling2D(size=pool_size),
            layers.Convolution2D(lev, kernel_size, padding='same', activation=None)
            ]
            )
    
    
    optimizer= keras.optimizers.Adam(lr=lr)

    if N_gpu > 1:
        with tf.device("/cpu:0"):
            # convert the model to a model that can be trained with N_GPU GPUs
             model = keras.utils.multi_gpu_model(model, gpus=N_gpu)
             
    model.compile(loss='mean_squared_error', optimizer = optimizer)
    
    return model

In [17]:
##Make prediction
for lead_time in range(11,12):
    for i in range (0,480,1):
        X_train,y_train, X_test, y_test = prepare_data(lead_time)

        ## The tuning process produced the hyperparameters below.
        params = {'conv_depth': 32, 'hidden_size': 500,
                  'kernel_size': 4, 'lr': 3e-05, 'n_hidden_layers': 0}


        print(params)
        param_string = '_'.join([str(e) for e in (N_train,num_epochs,lead_time)])


        #run the model
        model = build_model(**params)

        print(model.summary())


        print('start training')
        hist = model.fit(X_train, y_train,
                           batch_size = batch_size,
                 verbose=1, 
                 epochs = num_epochs,
                 validation_data=(X_test,y_test),
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                            min_delta=0,
                                            patience=5, #To ensure a great deal of patience before ending
                                            verbose=0, mode='auto'),
                           keras.callbacks.ModelCheckpoint('best_weights.h5', monitor='val_loss', 
                                                        verbose=1, save_best_only=True, 
                                                        save_weights_only=True, mode='auto', period=1)]
                 )

        print('finished training')
        
        # To ensure the best model performance from the training based on learning curve,
        # Because the early stopping callback saves the model's "patience" epochs after the best one, this is required.
        model.load_weights('best_weights.h5')

        # delete the file generated by Model Checkppoint
        os.system('rm best_weights.h5')


        model.save_weights(outdir+'weights_tunedparams_leadtime'+str(lead_time)+'train_'+str(i)+'params_'+param_string+'.h5')


        # reformat history

        hist =  hist.history


        y_test_predicted = model.predict(X_test)

        # compute accuracy
        res = []
        rmse = np.sqrt(np.mean((y_test_predicted - y_test)**2))
        acc = acc_score(y_test_predicted, y_test)

        res.append(dict(hist=hist,params=params, scores=[rmse,acc]))

        pickle.dump(res,open(outdir+'training_result_sequential'+param_string+'.pkl','wb'))

        #plot loss value
        plt.figure()
        plt.plot(hist['val_loss'], label='val_loss')
        plt.plot(hist['loss'], label='train loss')

        plt.legend()
        
        #save loss to .png
        plt.savefig(outdir+'cwcnn_history_tunedparams_leadtime'+str(lead_time)+str(i)+'.png')

        pd.DataFrame(hist).to_csv(outdir+'history_tunedparams_leadtime'+str(lead_time)+str(i)+'.csv')


        '''y_test is a xarray dataarray, but y_test_predicted is now a numpy array.
        Therefore, we convert it to an xarray with exactly the same coordinates and dims by using dim from y_test.'''
        # save the validation
        y_test_size= xr.open_dataarray(testsizepath+'y_testsize_'+str(i)+'_32_64_1.nc')

        y_test_predicted_new = xr.DataArray(data=y_test_predicted, coords=y_test_size.coords, dims=y_test_size.dims)
        y_test_new = xr.DataArray(data=y_test, coords=y_test_size.coords, dims=y_test_size.dims)

        # save the predictions
        y_test_predicted_new.to_netcdf(outdir+'/predictions_tuned_leadtime'+str(lead_time)+'train_'+str(N_train)+'month_'+str(i)+'params_'+param_string+'.nc')
        y_test_new.to_netcdf(outdir+'/truevalues_tuned_leadtime'+str(lead_time)+'train_'+str(N_train)+'month_'+str(i)+'params_'+param_string+'.nc')

{'conv_depth': 32, 'hidden_size': 500, 'kernel_size': 4, 'lr': 3e-05, 'n_hidden_layers': 0}
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 32, 64, 32)        544       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 32, 64, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 32, 64, 32)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 64, 32)        16416     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 32, 64, 32)       0         
 2D)                                                             
                            

C:\Users\user\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


45/45 [==============================] - ETA: 0s - loss: 0.2255
Epoch 1: val_loss improved from inf to 0.10555, saving model to best_weights.h5
45/45 [==============================] - 19s 407ms/step - loss: 0.2255 - val_loss: 0.1055
Epoch 2/10
45/45 [==============================] - ETA: 0s - loss: 0.0748
Epoch 2: val_loss improved from 0.10555 to 0.05288, saving model to best_weights.h5
45/45 [==============================] - 17s 388ms/step - loss: 0.0748 - val_loss: 0.0529
Epoch 3/10
45/45 [==============================] - ETA: 0s - loss: 0.0344
Epoch 3: val_loss improved from 0.05288 to 0.02170, saving model to best_weights.h5
45/45 [==============================] - 18s 392ms/step - loss: 0.0344 - val_loss: 0.0217
Epoch 4/10
45/45 [==============================] - ETA: 0s - loss: 0.0158
Epoch 4: val_loss improved from 0.02170 to 0.01072, saving model to best_weights.h5
45/45 [==============================] - 17s 372ms/step - loss: 0.0158 - val_loss: 0.0107
Epoch 5/10
45/45 [=

C:\Users\user\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


45/45 [==============================] - ETA: 0s - loss: 0.2322
Epoch 1: val_loss improved from inf to 0.10880, saving model to best_weights.h5
45/45 [==============================] - 17s 374ms/step - loss: 0.2322 - val_loss: 0.1088
Epoch 2/10
45/45 [==============================] - ETA: 0s - loss: 0.0736
Epoch 2: val_loss improved from 0.10880 to 0.04984, saving model to best_weights.h5
45/45 [==============================] - 17s 369ms/step - loss: 0.0736 - val_loss: 0.0498
Epoch 3/10
45/45 [==============================] - ETA: 0s - loss: 0.0354
Epoch 3: val_loss improved from 0.04984 to 0.02355, saving model to best_weights.h5
45/45 [==============================] - 16s 366ms/step - loss: 0.0354 - val_loss: 0.0235
Epoch 4/10
45/45 [==============================] - ETA: 0s - loss: 0.0187
Epoch 4: val_loss improved from 0.02355 to 0.01346, saving model to best_weights.h5
45/45 [==============================] - 17s 384ms/step - loss: 0.0187 - val_loss: 0.0135
Epoch 5/10
45/45 [=

...

45/45 [==============================] - ETA: 0s - loss: 0.2248
Epoch 1: val_loss improved from inf to 0.11261, saving model to best_weights.h5
45/45 [==============================] - 27s 580ms/step - loss: 0.2248 - val_loss: 0.1126
Epoch 2/10
45/45 [==============================] - ETA: 0s - loss: 0.0618
Epoch 2: val_loss improved from 0.11261 to 0.04239, saving model to best_weights.h5
45/45 [==============================] - 26s 568ms/step - loss: 0.0618 - val_loss: 0.0424
Epoch 3/10
45/45 [==============================] - ETA: 0s - loss: 0.0261
Epoch 3: val_loss improved from 0.04239 to 0.02045, saving model to best_weights.h5
45/45 [==============================] - 26s 583ms/step - loss: 0.0261 - val_loss: 0.0205
Epoch 4/10
45/45 [==============================] - ETA: 0s - loss: 0.0122
Epoch 4: val_loss improved from 0.02045 to 0.01141, saving model to best_weights.h5
45/45 [==============================] - 26s 569ms/step - loss: 0.0122 - val_loss: 0.0114
Epoch 5/10
45/45 [=

C:\Users\user\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


45/45 [==============================] - ETA: 0s - loss: 0.3015
Epoch 1: val_loss improved from inf to 0.22561, saving model to best_weights.h5
45/45 [==============================] - 27s 598ms/step - loss: 0.3015 - val_loss: 0.2256
Epoch 2/10
45/45 [==============================] - ETA: 0s - loss: 0.1092
Epoch 2: val_loss improved from 0.22561 to 0.07294, saving model to best_weights.h5
45/45 [==============================] - 27s 594ms/step - loss: 0.1092 - val_loss: 0.0729
Epoch 3/10
45/45 [==============================] - ETA: 0s - loss: 0.0508
Epoch 3: val_loss improved from 0.07294 to 0.03668, saving model to best_weights.h5
45/45 [==============================] - 27s 591ms/step - loss: 0.0508 - val_loss: 0.0367
Epoch 4/10
45/45 [==============================] - ETA: 0s - loss: 0.0262
Epoch 4: val_loss improved from 0.03668 to 0.02039, saving model to best_weights.h5
45/45 [==============================] - 26s 585ms/step - loss: 0.0262 - val_loss: 0.0204
Epoch 5/10
45/45 [=

C:\Users\user\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


45/45 [==============================] - ETA: 0s - loss: 0.2906
Epoch 1: val_loss improved from inf to 0.19438, saving model to best_weights.h5
45/45 [==============================] - 27s 602ms/step - loss: 0.2906 - val_loss: 0.1944
Epoch 2/10
45/45 [==============================] - ETA: 0s - loss: 0.0764
Epoch 2: val_loss improved from 0.19438 to 0.02848, saving model to best_weights.h5
45/45 [==============================] - 26s 587ms/step - loss: 0.0764 - val_loss: 0.0285
Epoch 3/10
45/45 [==============================] - ETA: 0s - loss: 0.0185
Epoch 3: val_loss improved from 0.02848 to 0.01212, saving model to best_weights.h5
45/45 [==============================] - 27s 592ms/step - loss: 0.0185 - val_loss: 0.0121
Epoch 4/10
45/45 [==============================] - ETA: 0s - loss: 0.0098
Epoch 4: val_loss improved from 0.01212 to 0.00738, saving model to best_weights.h5
45/45 [==============================] - 27s 592ms/step - loss: 0.0098 - val_loss: 0.0074
Epoch 5/10
45/45 [=

In [18]:
##Read the accuracy
import pickle
import json

import numpy as np

res = pickle.load(open(outdir+'training_result_sequential1188_10_0.pkl','rb'))

# get each run's final validating loss

final_val_losses = [e['hist']['val_loss'][-1] for e in res]

# convert to array
final_val_losses = np.array(final_val_losses)

# find the smallest index.

idx_smallest = np.argmin(final_val_losses)


res_best = res[idx_smallest]

print(res_best)

with open('training_result.txt','w') as f :
    f.write(str(res_best))

{'hist': {'loss': [0.21433058381080627, 0.06211373209953308, 0.03262367472052574, 0.016888760030269623, 0.011370740830898285, 0.008568724617362022, 0.006891950033605099, 0.0057090274058282375, 0.004772605374455452, 0.004051714204251766], 'val_loss': [0.10516417026519775, 0.046481866389513016, 0.023316631093621254, 0.014155803248286247, 0.010411395691335201, 0.008196555078029633, 0.006777796428650618, 0.005658963229507208, 0.004787841811776161, 0.004108403343707323]}, 'params': {'conv_depth': 32, 'hidden_size': 500, 'kernel_size': 4, 'lr': 3e-05, 'n_hidden_layers': 0}, 'scores': [0.06409682, 0.9802483707924725]}


# Prodicting PDO

In [13]:
#import the average trend of global SST 

gsst = xr.open_dataarray('C:/Users/user/Research/Research Code/CNN/Data/'+'ersst_1854_2022_mech.nc')
gclm = gsst.groupby('time.month').mean(dim='time')
ganm = (gsst.groupby('time.month') - gclm)
mglobal=ganm.mean(("lon", "lat"), skipna=True)
plt.plot(mglobal)
plt.show

<function matplotlib.pyplot.show(*, block=None)>

In [14]:
##Setup validation period
ystr = 1983
yend = 2022

##setup dimension
lead_time = 359
N_train = 1428

##Imported ensemble training results used to transform land mask
ncin = Dataset(ifile+'combine_train_1428_lead_11.nc')


sst  = ncin.variables['__xarray_dataarray_variable__'][:,:-9,:] 
time  = ncin.variables['time'][:]
lat  = ncin.variables['lat'][:-9]
lon  = ncin.variables['lon'][:]
lev  = ncin.variables['lev'][:]
ncin.close()

nt,nlat,nlon,nlev = sst.shape    
print(sst.shape )

## Imported ensemble pred data
ds = xr.open_dataset(ifile+'combine_train_1428_lead_11.nc')

ds = ds.sel(lat=slice(70, 20), lon=slice(122, 250))
print(ds)

(480, 23, 64, 1)
<xarray.Dataset>
Dimensions:                        (time: 480, lon: 64, lat: 23, lev: 1)
Coordinates:
  * time                           (time) datetime64[ns] 1983-01-01 ... 2022-...
  * lon                            (lon) float32 122.0 124.0 ... 246.0 248.0
  * lat                            (lat) float32 64.0 62.0 60.0 ... 22.0 20.0
  * lev                            (lev) float64 0.0
Data variables:
    __xarray_dataarray_variable__  (time, lat, lon, lev) float32 ...
Attributes:
    CDI:          Climate Data Interface version 2.0.4 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Sat Aug 12 14:12:32 2023: cdo mergetime predictions_tuned_l...
    CDO:          Climate Data Operators version 2.0.4 (https://mpimet.mpg.de...


In [18]:
##import land mask
lmfile = 'data\lsmask.nc'
lmset  = Dataset(lmfile)
lsmask = lmset['mask'][0,24:70:2,122:250:2]# read land mask
lsmask = lsmask-1

num_repeats = nt
lsm = np.stack([lsmask]*num_repeats,axis=-1).transpose((2,0,1))
lsm.shape

(480, 23, 64)

In [16]:
a = np.arange('1983-01', '2023-01', dtype='datetime64[M]')

In [17]:
# === Climatology and Anomalies 
sst = ds.__xarray_dataarray_variable__
clm = sst.sel(time=slice(f'{ystr}-01-01',f'{yend}-12-01')).groupby('time.month').mean(dim='time')
anm = (sst.groupby('time.month') - clm)

# remove lev dimension
anm=anm.isel(lev=0)

##Prediction
#detrend
dpdo = detrend_dim(anm,mglobal,'time',1)

#Mask out land
sst_diff = dpdo
sst_diff = np.ma.masked_array(sst_diff, mask=lsm)
sst_diff[lsm<0] = np.nan
#sst_diff=sst_diff.to_numpy()
print(sst_diff.shape)

(480, 23, 64)


In [19]:
#To perform the EOF analysis, create an EOF solver.
#Prior to computing the EOFs, latitude weights are applied with a cosine.
wgts   = np.sqrt(np.cos(np.deg2rad(lat)))
wgts   = wgts.reshape(len(wgts), 1)
wgts.shape

#Obtained the leading EOFs
solver = Eof(sst_diff, weights=wgts)

eof1 = solver.eofs(neofs=1)
pc1  = solver.pcs(npcs=1, pcscaling=1)
varfrac = solver.varianceFraction()
lambdas = solver.eigenvalues()
Covariance_eof1 = solver.eofsAsCovariance(neofs=1, pcscaling=0)

leading_eof = solver.eofsAsCorrelation(neofs=1)
eigenvalue1 = solver.eigenvalues(neigs=1)

In [20]:
#Creat PDO index
pc=pc1 [:,]
pc_mean=pc1[:,].mean()
pc_std=pc1[:,].std()

#Normalised
pcs = xr.apply_ufunc(

lambda x, m, s: x  / s,
pc,
pc_mean,
pc_std, dask = 'allowed', vectorize = True)
pcs.shape
PDO=pcs[:,0]

(480, 1)

In [26]:
#Smoothed with 10-year low pass
cutoff = 0.05
lw_pred = lowpass_filter(cutoff ,PDO)

np.savetxt(outdir_smooth_pred+'Result_PDO index_pred_cwcnn_smoothed'+str(lead_time)+'.csv',lw_pred, delimiter=',', fmt='%s')

##Imported and converted predicted data
PDO_cnn = np.loadtxt(outdir_smooth_pred+'Result_PDO index_pred_cwcnn_smoothed'+str(lead_time)+'.csv', dtype=float)
PDO_pred = adjust(PDO_cnn)


##Import validation data from NCEI PDO index from 1983-2022 
#https://www.ncei.noaa.gov/access/monitoring/pdo/

PDO_ncei = pd.read_excel(ifile_data+'NCEI PDO index _overall.xlsx',header=None,index_col=None,
                      skiprows=985, skipfooter = 0,
                      usecols=[1]  
                     )

PDO_ncei = adjust(PDO_ncei)
lw_ncei = lowpass_filter(cutoff ,PDO_ncei)
PDO_ncei=adjust(lw_ncei)

In [27]:
##Plot EOF map and PC Amplitude
parallels = np.arange(-90,90,30.)
meridians = np.arange(-180,180,30)

for i in range(0,1):
    fig = plt.figure(figsize=(3,3))
    plt.subplot(211)
    
    m = Basemap(projection='cyl', llcrnrlon=min(lon), llcrnrlat=min(lat), urcrnrlon=max(lon), urcrnrlat=max(lat))   
    x, y = m(*np.meshgrid(lon, lat))
    clevs = np.linspace(np.min(eof1[i,:,:].squeeze()), np.max(eof1[i,:,:].squeeze()), 21)
    cs = m.contourf(x, y, eof1[i,:,:].squeeze(), clevs, cmap=plt.cm.RdBu_r)
    m.drawcoastlines()  
    m.drawparallels(parallels, labels=[1,0,0,0])
    m.drawmeridians(meridians, labels=[1,0,0,1])

    cb = m.colorbar(cs, 'right', size='5%', pad='2%')
    cb.set_label('EOF', fontsize=12)
    plt.title('EOF ' + str(i+1), fontsize=16)

    plt.subplot(212)
    days = np.linspace(1983,2022,nt)
    plt.plot(days, pc1[:,i], linewidth=2)
    plt.axhline(0, color='k')
    plt.xlabel('Year')
    plt.ylabel('PC Amplitude')   
    plt.ylim(np.min(pc1.squeeze()), np.max(pc1.squeeze()))

In [32]:
##Calculate the average correlation
Correlation_cnn = []
for i in range (1,481): 
    correlation(PDO_pred,PDO_ncei,Correlation_cnn)

print('Correlation: ', mean(Correlation_cnn[1:481]))

Correlation:  0.86


# END